In [2]:
import numpy as np
from numba import jit, njit
from statsmodels.tsa.stattools import acf

In [ ]:
# ACD - Parametrik Otokorelasyon Fonksiyonu
@njit
def pcf(param:np.array, dimc:int=30) -> np.array:

    a,b = param
    ab = a*b
    out = np.zeros(dimc+1)
    out[1] = a + (ab*a) / (1 - 2*ab - b**2)
    i=2
    while i<=dimc:
        out[i] = out[1] * (a+b)**(i-1)
        i+=1
    return out[1:]

#======================================================
# Bartlett formülü
@njit
def Bartlett_C(scor:np.array, nobs:int, dimc:int=30):
    # -------------------------------------------------
    C = np.zeros((dimc, dimc))
    for i in range(dimc):
        for j in range(i, dimc):
            ii, jj = i + 1, j + 1
            cij = 0.0
            for k in range(1, nobs - dimc):
                _2rk = 2 * scor[k]
                kdi, kdj = abs(k - ii), abs(k - jj)
                a = scor[kdi] + scor[k + ii] - _2rk * scor[ii]
                b = scor[kdj] + scor[k + jj] - _2rk * scor[jj]
                cij += a * b
    # -------------------------------------------------
            C[i, j] = C[j, i] = cij
    # -------------------------------------------------
    return C
#======================================================
# Newey-West formülü
@njit
def NeweyWest_C(pcor:np.array, data:np.array, dimc:int=30):

    mxh = data.size - dimc
    #--------------------------------------------------
    D = data[::-1] - data.mean() #
    #--------------------------------------------------
    h = np.zeros((mxh, dimc))
    t=0
    while t<mxh:
        i = t+1
        h[t] = D[i:dimc+i]-pcor*D[t]
        h[t]*= D[t]
        t = i
    #--------------------------------------------------
    Bw = 1 - np.arange(dimc+1)/(dimc+1)
    C = np.dot(h.T, h)
    i=1
    while i<=dimc:
        omg = np.dot(h[i:].T, h[:-i])
        C += Bw[i] * (omg + omg.T)
        i+=1
    #--------------------------------------------------
    return C
#======================================================
@jit(forceobj=True)
def MDE_B(param:np.array, data:np.array, dimc:int=30) -> float:
    #--------------------------------------------------
    α, β = param
    n = data.size
    # kısıtlar ----------------------------------------
    if (α <= 0.0) or (β <= 0.0) or (1-α-β <= 0.0):
        return 1e12
    #--------------------------------------------------
    pr = pcf(param, dimc)
    sr = acf(data, nlags=n)
    #--------------------------------------------------
    C  = Bartlett_C(sr, n, dimc)
    #--------------------------------------------------
    R  = sr[1:dimc+1] - pr
    #--------------------------------------------------
    return R.T @ np.inv(C) @ R

#======================================================
@jit(forceobj=True)
def MDE_NW(param:np.array, data:np.array, dimc:int=30) -> float:
    #--------------------------------------------------
    α, β = param
    # kısıtlar ----------------------------------------
    if (α <= 0.0) or (β <= 0.0) or (1-α-β <= 0.0):
        return 1e12
    #--------------------------------------------------
    pr = pcf(param, dimc)
    sr = acf(data, nlags=dimc)[1:]
    #--------------------------------------------------
    R  = sr - pr
    #--------------------------------------------------
    C = NeweyWest_C(pr, data, dimc)
    #--------------------------------------------------
    return R.T @ inv(C) @ R